In [1]:
drug='EWV'
complex=drug

In [2]:
import os
ff_folder  = os.path.realpath("../../../smsl/opt/force_field")
mdp_folder = os.path.realpath("../../../smsl/opt/mdp")

### Copy Drug From Last Folder

In [3]:
!cp -rv ../1.GenTopology/{drug}.amb2gmx/*{drug}* ./ ## .gro .top .itp

'../1.GenTopology/EWV.amb2gmx/EWV_GMX.gro' -> './EWV_GMX.gro'
'../1.GenTopology/EWV.amb2gmx/EWV_GMX.top' -> './EWV_GMX.top'
'../1.GenTopology/EWV.amb2gmx/posre_EWV.itp' -> './posre_EWV.itp'


### Modify `POH_GMX.top`

In [4]:
!mv {drug}_GMX.top drug_GMX.top
# 1. 將[nbfunc, Compound, ]關鍵字的下一行註解
!sed -i '/nbfunc/{N;s/\n/\n;/}' drug_GMX.top
!sed -i '/Compound/{N;s/\n/\n;/}' drug_GMX.top
# 2. 將 [atomtypes] 內 name bond_type的文字轉小寫
!sed -E '/\[ atomtypes \]/,/^$/ {\
    /^[[:space:]]*[A-Z]/ {\
        s/^([[:space:]]*)([A-Z0-9*]+)([[:space:]]+)([A-Z0-9*]+)/\1\L\2\3\L\4/\
    }\
}' drug_GMX.top > temp.top && mv temp.top drug_GMX.top
# 3. 將 [atoms] 內 type的文字轉小寫
!sed -E '/\[ atoms \]/,/^$/ {\
    /^[[:space:]]*[0-9]/ {\
        s/^([[:space:]]*[0-9]+[[:space:]]+)([A-Z0-9*]+)([[:space:]]+)/\1\L\2\3/\
    }\
}' drug_GMX.top > temp.top && mv temp.top drug_GMX.top
!mv drug_GMX.top {drug}_GMX.top

In [5]:
force_field = 'amber14sb_parmbsc1.ff'

top_data = f'''\
; Include forcefield parameters
#include "{ff_folder}/{force_field}/forcefield.itp"

; Include ligand parameters
#include "{drug}_GMX.top"

; Include ligand topology
#ifdef POSRES
#include "posre_{drug}.itp"
#endif

; Include water topology
#include "{ff_folder}/{force_field}/tip3p.itp"

#ifdef POSRES_WATER
; Position restraint for each water oxygen
[ position_restraints ]
;  i funct       fcx        fcy        fcz
   1    1       1000       1000       1000
#endif

; Include topology for ions
#include "{ff_folder}/{force_field}/ions.itp"

[ system ]
; Name
Protein in water

[ molecules ]
; Compound        #mols
{drug}               1\
'''

!echo '{top_data}' > topol.top

In [6]:
!gmx editconf -f {drug}_GMX.gro -o {complex}.gro.pdb

                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx editconf -f EWV_GMX.gro -o EWV.gro.pdb

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 43 atoms
Volume: 2692.35 nm^3, corresponds to roughly 1211500 electrons
No velocities found

GROMACS reminds you: "If You're So Special Why aren't You Dead ?" (The Breeders)



### Add box and add solvent (water)

In [7]:
!gmx editconf -f {complex}.gro.pdb -o {complex}_dode.pdb -c yes -d 0.865 -bt dodecahedron

                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx editconf -f EWV.gro.pdb -o EWV_dode.pdb -c yes -d 0.865 -bt dodecahedron

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 43 atoms
Volume: 2692.35 nm^3, corresponds to roughly 1211500 electrons
No velocities found
    system size :  1.439  0.761  0.308 (nm)
    diameter    :  1.447               (nm)
    center      :  0.036  0.007 -0.003 (nm)
    box vectors : 28.788 15.212  6.148 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :2692.35               (nm^3)
    shift       :  2.347  2.376  1.126 (nm)
new center      :  2.383  2.383  1.123 (nm)
new box vectors :  3.177  3.177  3.177 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume 

### Add solvent (water)

In [8]:
!gmx solvate -cp {complex}_dode.pdb -cs spc216.gro -o {complex}_water.pdb -p topol.top

                     :-) GROMACS - gmx solvate, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx solvate -cp EWV_dode.pdb -cs spc216.gro -o EWV_water.pdb -p topol.top

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be diff

## Remove PBC and See dodecahedron
select `0(System)` for output group

In [9]:
!echo "0" | gmx trjconv -s {complex}_water.pdb -f {complex}_water.pdb -o {complex}_water.dode.pdb -pbc res -ur compact

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx trjconv -s EWV_water.pdb -f EWV_water.pdb -o EWV_water.dode.pdb -pbc res -ur compact

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write pdb: Protein data bank file

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.


         that are broken across periodic boundaries, they
         canno

In [10]:
## visualize
#!vmd {complex}_water.dode.pdb
#!pymol {complex}_water.dode.pdb

### Add ions
gmx genion ~
`-conc` unit: (mol/liters) default=0.125  
sele `6(SOL)` for group of solvent molecules

In [11]:
!gmx grompp -f {mdp_folder}/ions.mdp -c {complex}_water.pdb -p topol.top -o ions.tpr

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx grompp -f /home/user/workfolder/g4dna_protocol_demo/smsl/opt/mdp/ions.mdp -c EWV_water.pdb -p topol.top -o ions.tpr

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file /home/user/workfolder/g4dna_protocol_demo/smsl/opt/mdp/ions.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to -560121093

Generated 4753 of the 4753 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4753 of the 4753 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'EWV'

Excluding 2 bonded neighbours molecule type 'SOL'

NOTE 2 [file topol.top, line 32]

In [12]:
!echo "SOL" | gmx genion -s ions.tpr -o {complex}_solv.pdb -p topol.top -pname K -nname CL -neutral -conc 0.125

                      :-) GROMACS - gmx genion, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx genion -s ions.tpr -o EWV_solv.pdb -p topol.top -pname K -nname CL -neutral -conc 0.125

Reading file ions.tpr, VERSION 2023.2 (single precision)
Reading file ions.tpr, VERSION 2023.2 (single precision)
Will try to add 2 K ions and 3 CL ions.
Select a continuous group of solvent molecules
Group     0 (         System) has  2167 elements
Group     1 (          Other) has    43 elements
Group     2 (            EWV) has    43 elements
Group     3 (          Water) has  2124 elements
Group     4 (            SOL) has  2124 elements
Group     5 (      non-Water) has    43 elements
Select a group: Selected 4: 'SOL'
Number of (3-atomic) solvent molecules: 708

Processing topology
Replacing 5 solute molecules in topology file (top

In [13]:
## visualize
!echo "0" | gmx trjconv -s {complex}_solv.pdb -f {complex}_solv.pdb -o {complex}_solv.dode.pdb -pbc res -ur compact
#!pymol {complex}_solv.dode.pdb

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx trjconv -s EWV_solv.pdb -f EWV_solv.pdb -o EWV_solv.dode.pdb -pbc res -ur compact

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write pdb: Protein data bank file

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.


         that are broken across periodic boundaries, they
         cannot b

### Energy Minimization

In [14]:
!gmx grompp -f {mdp_folder}/min1.mdp -c {complex}_solv.pdb -r {complex}_solv.pdb -p topol.top -o min1.tpr
!gmx mdrun -deffnm min1 -c min1.pdb

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx grompp -f /home/user/workfolder/g4dna_protocol_demo/smsl/opt/mdp/min1.mdp -c EWV_solv.pdb -r EWV_solv.pdb -p topol.top -o min1.tpr

Setting the LD random seed to -687867921

Generated 4753 of the 4753 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4753 of the 4753 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'EWV'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bonded neighbours molecule type 'K'

Excluding 1 bonded neighbours molecule type 'CL'
Analysing residue names:
There are:     1      Other residues
There are:   703      Water residues
There are:     5        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and sp

In [15]:
## -r argument min1.pdb or {complex}_solv.pdb
!gmx grompp -f {mdp_folder}/min2.mdp -c min1.pdb -r {complex}_solv.pdb -p topol.top -o min2.tpr
!gmx mdrun -deffnm min2 -c min2.pdb

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx grompp -f /home/user/workfolder/g4dna_protocol_demo/smsl/opt/mdp/min2.mdp -c min1.pdb -r EWV_solv.pdb -p topol.top -o min2.tpr


NOTE 1 [file /home/user/workfolder/g4dna_protocol_demo/smsl/opt/mdp/min2.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to -1293441129

Generated 4753 of the 4753 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4753 of the 4753 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'EWV'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into 

## Make Index File

In [16]:
cmd = '''\
q
'''
# 1 | 3
# name 10 DNA_drug
# a 1-717
# name 11 DNA_2K
# a 1-717 | 3
# name 12 DNA_2K_drug
!echo -e "{cmd}" | gmx make_ndx -f min2.pdb -o {complex}.simulate.ndx

                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_ewv/b.amber.drug_ff/3.Run_md
Command line:
  gmx make_ndx -f min2.pdb -o EWV.simulate.ndx


Reading structure file
Going to read 0 old index file(s)
Analysing residue names:
There are:     1      Other residues
There are:   703      Water residues
There are:     5        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              :  2157 atoms
  1 Other               :    43 atoms
  2 EWV                 :    43 atoms
  3 K                   :     2 atoms
  4 CL                  :     3 atoms
  5 Water               :  2109 atoms
  6 SOL                 :  2109 atoms
  7 non-Water           :    48 atoms
  8 Ion                 :     5 atoms
  9 Water_and_ions      :  2114 atoms

 nr : group      '!': not 

### Heating
(In the energy minimization process, the only parameter required is the initial structure. However, for NVT or NPT processes, since the .pdb file does not contain information about speed or temperature state, an additional parameter, "-t", must be included to provide this information.)

In [17]:
print(f"gmx grompp -f {mdp_folder}/heat.mdp -c min2.pdb -r {complex}_solv.pdb -p topol.top -o heat.tpr -n {complex}.simulate.ndx")
print(f"gmx mdrun -deffnm heat -c heat.pdb -v")

gmx grompp -f /home/user/workfolder/g4dna_protocol_demo/smsl/opt/mdp/heat.mdp -c min2.pdb -r EWV_solv.pdb -p topol.top -o heat.tpr -n EWV.simulate.ndx
gmx mdrun -deffnm heat -c heat.pdb -v


### Equilibration (NVT)

In [18]:
print(f"gmx grompp -f {mdp_folder}/nvt1.mdp -c heat.pdb -r {complex}_solv.pdb -t heat.cpt -p topol.top -o nvt1.tpr -n {complex}.simulate.ndx")
print(f"gmx mdrun -deffnm nvt1 -c nvt1.pdb -v")

gmx grompp -f /home/user/workfolder/g4dna_protocol_demo/smsl/opt/mdp/nvt1.mdp -c heat.pdb -r EWV_solv.pdb -t heat.cpt -p topol.top -o nvt1.tpr -n EWV.simulate.ndx
gmx mdrun -deffnm nvt1 -c nvt1.pdb -v


### Equilibration (NPT)

In [19]:
print(f"gmx grompp -f {mdp_folder}/npt1.mdp -c nvt1.pdb -r {complex}_solv.pdb -t nvt1.cpt -p topol.top -o npt1.tpr -n {complex}.simulate.ndx")
print(f"gmx mdrun -deffnm npt1 -c npt1.pdb -v")

gmx grompp -f /home/user/workfolder/g4dna_protocol_demo/smsl/opt/mdp/npt1.mdp -c nvt1.pdb -r EWV_solv.pdb -t nvt1.cpt -p topol.top -o npt1.tpr -n EWV.simulate.ndx
gmx mdrun -deffnm npt1 -c npt1.pdb -v


In [20]:
print(f"gmx grompp -f {mdp_folder}/npt2.mdp -c npt1.pdb -r {complex}_solv.pdb -t npt1.cpt -p topol.top -o npt2.tpr -n {complex}.simulate.ndx")
print(f"gmx mdrun -deffnm npt2 -c npt2.pdb -v")

gmx grompp -f /home/user/workfolder/g4dna_protocol_demo/smsl/opt/mdp/npt2.mdp -c npt1.pdb -r EWV_solv.pdb -t npt1.cpt -p topol.top -o npt2.tpr -n EWV.simulate.ndx
gmx mdrun -deffnm npt2 -c npt2.pdb -v


### Run MD (test)

In [21]:
!echo "gmx grompp -f md0.mdp -c npt2.pdb -p topol.top -o md0.tpr -n {complex}.simulate.ndx"
!echo "gmx mdrun -deffnm md0 -c md0.pdb"

gmx grompp -f md0.mdp -c npt2.pdb -p topol.top -o md0.tpr -n EWV.simulate.ndx
gmx mdrun -deffnm md0 -c md0.pdb
